In [1]:
# 크롤링 작업을 위한 라이브러리 임포트
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import requests
import time   # 코드 진행 지연을 위한 time 임포트
from selenium.webdriver.common.by import By   # 2022년 7월 이후 selenium 업데이트로 인한 xpath 추적 임포트
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [12]:
region = str(input("지역 : "))
page  = int(input("페이지 : "))

driver = webdriver.Chrome("chromedriver")
time.sleep(1)
driver.get('https://map.kakao.com/')
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]').send_keys('%s 식당\n' % region)
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="info.search.place.sort"]/li[1]/a').send_keys(Keys.ENTER)
time.sleep(2)

n_list = []
s_list = []
a_list = []
ka_dict = {}




## 페이지 5보다 작을 때
if page < 5:
    for j in range(1,page+1):
        source = driver.page_source
        parsed_source = bs(source, 'html.parser')

        names_div_list = parsed_source.find_all("li",class_ = 'PlaceItem clickArea')        # 가게명
        names_span_list = parsed_source.find_all('span', class_ = 'screen_out')
        span_names = names_span_list[1:31:2]
        for i in range(len(span_names)):
            names = span_names[i].text
            n_list.append(names)


        div_scores = parsed_source.find_all('em', class_ = "num")                           # 점수
        del div_scores[-1]
        for i in range(len(div_scores)):
            scores = div_scores[i].text
            s_list.append(scores)


        div_address = parsed_source.find_all('div', class_ = "addr")                        # 주소
        for i in range(len(div_address)):
            address = div_address[i].text.replace('\n','').split('(지번)')[0]
            a_list.append(address)
            
        driver.find_element(By.XPATH,'//*[@id="info.search.page.no%s"]' % (j+1)).send_keys(Keys.ENTER)        # 페이지 넘기기
        time.sleep(2)
        
        
        
        
        
        

## 각 묶음 페이지의 1쪽 해당
for j in range(page//5):
    source = driver.page_source
    parsed_source = bs(source, 'html.parser')

    names_div_list = parsed_source.find_all("li",class_ = 'PlaceItem clickArea')              
    names_span_list = parsed_source.find_all('span', class_ = 'screen_out')
    span_names = names_span_list[1:31:2]
    for i in range(len(span_names)):
        names = span_names[i].text
        n_list.append(names)

    
    div_scores = parsed_source.find_all('em', class_ = "num")
    del div_scores[-1]
    for i in range(len(div_scores)):
        scores = div_scores[i].text
        s_list.append(scores)
    
    
    div_address = parsed_source.find_all('div', class_ = "addr")
    for i in range(len(div_address)):
        address = div_address[i].text.replace('\n','').split('(지번)')[0]
        a_list.append(address)
    time.sleep(2)
    
## 각 묶음 페이지의 2, 3, 4, 5쪽 해당
    for i in range(2,7):
        time.sleep(1)
        if i == 6:
            if page%5 == 0:
                continue
            driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)
            time.sleep(2)
        else:
            driver.find_element(By.XPATH,'//*[@id="info.search.page.no%s"]' % i).send_keys(Keys.ENTER)
            time.sleep(2)

        source = driver.page_source
        parsed_source = bs(source, 'html.parser')

        names_div_list = parsed_source.find_all("li",class_ = 'PlaceItem clickArea')
        names_span_list = parsed_source.find_all('span', class_ = 'screen_out')
        span_names = names_span_list[1:31:2]
        for i in range(len(span_names)):
            names = span_names[i].text
            n_list.append(names)


        div_scores = parsed_source.find_all('em', class_ = "num")
        del div_scores[-1]
        for i in range(len(div_scores)):
            scores = div_scores[i].text
            s_list.append(scores)


        div_address = parsed_source.find_all('div', class_ = "addr")
        for i in range(len(div_address)):
            address = div_address[i].text.replace('\n','').split('(지번)')[0]
            a_list.append(address)

        time.sleep(2)

## 입력한 페이지가 속한 묶음 페이지에 해당
if page > 5:
    for k in range(2,page%5+1):

        driver.find_element(By.XPATH,'//*[@id="info.search.page.no%s"]' % k).send_keys(Keys.ENTER)
        time.sleep(2)

        source = driver.page_source
        parsed_source = bs(source, 'html.parser')

        names_div_list = parsed_source.find_all("li",class_ = 'PlaceItem clickArea')
        names_span_list = parsed_source.find_all('span', class_ = 'screen_out')
        span_names = names_span_list[1:31:2]
        for i in range(len(span_names)):
            names = span_names[i].text
            n_list.append(names)


        div_scores = parsed_source.find_all('em', class_ = "num")
        del div_scores[-1]
        for i in range(len(div_scores)):
            scores = div_scores[i].text
            s_list.append(scores)


        div_address = parsed_source.find_all('div', class_ = "addr")
        for i in range(len(div_address)):
            address = div_address[i].text.replace('\n','').split('(지번)')[0]
            a_list.append(address)

        time.sleep(2)


    
    
## 데이터 딕셔너리 형태로 적재
for i in range(len(n_list)):
    ka_dict[i] = (n_list[i], s_list[i], a_list[i])

print(ka_dict)

지역 : 경기도
페이지 : 9
{0: ('카페대너리스', '3.8', '경기 남양주시 조안면 북한강로 914'), 1: ('더티트렁크', '2.6', '경기 파주시 지목로 114'), 2: ('고기리막국수', '3.6', '경기 용인시 수지구 이종무로 157'), 3: ('레드브릿지', '3.3', '경기 파주시 광탄면 기산로 329 2층'), 4: ('가보정 1관', '3.2', '경기 수원시 팔달구 장다리로 282'), 5: ('본수원갈비 본점', '2.9', '경기 수원시 팔달구 중부대로223번길 41'), 6: ('보나리베카페', '4.1', '경기 남양주시 불암산로39번길 9-2 1,2층'), 7: ('어로프슬라이스피스', '3.1', '경기 용인시 처인구 백령로 47'), 8: ('오랑주리', '3.0', '경기 양주시 백석읍 기산로 423-19'), 9: ('장어의꿈', '3.5', '경기 남양주시 순화궁로 492-6 1층'), 10: ('반구정나루터집', '3.4', '경기 파주시 문산읍 반구정로85번길 13'), 11: ('리버레인', '4.2', '경기 가평군 청평면 북한강로 2141'), 12: ('테라로사 서종점', '3.7', '경기 양평군 서종면 북한강로 992'), 13: ('카페아를', '2.9', '경기 의정부시 동일로 204'), 14: ('파크프리베', '2.9', '경기 의정부시 동일로192번길 28-27'), 15: ('몬떼델피노', '3.3', '경기 김포시 양촌읍 김포한강6로 49-19'), 16: ('베이커리 씨어터', '3.3', '경기 남양주시 화도읍 경춘로2696번길 4-15 1동 1층'), 17: ('모아니', '', '경기 성남시 분당구 쇳골로 116 1층'), 18: ('이진상회', '3.6', '경기 이천시 마장면 서이천로 648 5동 1, 2층'), 19: ('일산칼국수 본점', '4.0', '경기 고양시 일산동구 경의로 467 1층'), 20: ('스페이스작', '3.1', '경기 부천시 까치로20번길

In [7]:
print(len(ka_dict))

75


In [8]:
# 최저점수 미만 걸러내기

selec = input("최저 점수(소수로 입력) : ")

for i in range(len(ka_dict)):
    if ka_dict[i][1] < selec:
        del ka_dict[i]
kakao = pd.DataFrame(ka_dict)

최저 점수(소수로 입력) : 3.0


In [10]:
kakao.T

,0,1,2
0,카페대너리스,3.8,경기 남양주시 조안면 북한강로 914
2,고기리막국수,3.6,경기 용인시 수지구 이종무로 157
3,레드브릿지,3.3,경기 파주시 광탄면 기산로 329 2층
4,가보정 1관,3.2,경기 수원시 팔달구 장다리로 282
6,보나리베카페,4.1,"경기 남양주시 불암산로39번길 9-2 1,2층"
...,...,...,...
70,수수카페,3.8,경기 양평군 양서면 북한강로89번길 16
71,청담추어정,3.9,경기 성남시 수정구 여수대로56번길 3
72,260도 부천점,4.1,경기 부천시 신흥로 178 두산위브더스테이트 3단지 216호
73,한성치킨,3.1,경기 김포시 양촌읍 석모로73번길 85 1층
